In [2]:
#testGetFullDF
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd
import yahoo_fin.stock_info as yf
import numpy as np
from stage import getStage
import pickle
from datetime import datetime

In [ ]:
#test_train_test_split
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
print(train)
print(test)


In [5]:
dfSorted = pd.read_pickle("stockData/nyseNasdaq/AAPL.pkl")
dfSorted.to_csv("sampleDfSorted.csv")

In [11]:
transactionFit = pd.read_pickle("transactionTemplate.pkl")
transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
transactionFitCopy = transactionFit
transactionFitCopy['total'] = 0
transactionFitCopy.to_csv("transactionTemplateCopy.csv")

In [1]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
    goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    index = 0
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                # print(individual[0][9],individual[0][10])
                if  df.at[date,'MA30Slope'] > 0 or df.at[date,'RS'] > 0.1:
                    # if df.at[date,'close'] > df.at[date,'MA30']:
                        # print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
            except:
                listOfSector.append(df.iat[-1,6]) 
                pass
            # print(listOfSector)
        goodSectorDf.iat[index,0] = set(listOfSector)
        index = index + 1
stockDataTickerList = pd.read_pickle("stockData/tickerList.pkl")
removeList = []
from stage import getStage
for ticker in stockDataTickerList:
    try:
        getStage(ticker,[0.9630183179676801, 1.3518854933368092, 0.16319993430933552, 1.0537971763840923, 1.006835620568861, 1.0103606174928899, 0.9033455933504492, 0.9731546594459972, 1.46426124905874, 0.9991612063815158, 0.09863994265206354, 0.9826811057391516, 0.9949684774619274, 0.8670538938494661, 0.9895837713208671],goodSectorDf)
    except:
        print(ticker)
        removeList.append(ticker)

DAVE
DPRO
FICV
GBNH
HCVIW
NHICW
PBAXW
PEGRW
PRLHW
RDIB
SAMAW
SANG
SCMAW
SMAPW
USCTW
VMGAW
ZIONP


In [2]:
for ticker in removeList:
    stockDataTickerList.remove(ticker)
for ticker in removeList:
    if ticker in stockDataTickerList:
        print("Broken")

In [3]:
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(stockDataTickerList, f)

In [2]:
getStage("AFRM",[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)

In [21]:
df = pd.read_pickle("transactionTemplate.pkl")
df.to_csv("transactionTemplate.csv")


In [6]:
now = datetime.now()
yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1).to_csv("AAPL.csv")

In [5]:
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            print(symbol)
            continue
        if not (df['close'] != 0).all() or df.empty:
            if df.empty:
                print(symbol)
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (initialPrice == 0): continue
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn


In [6]:
## Get list of returns of tickers
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
listOfDf = calculateGroupReturn(nasdaqList)

/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_48519/4014016260.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_48519/4014016260.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_48519/4014016260.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

CONE


/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_48519/4014016260.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_48519/4014016260.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_48519/4014016260.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

In [10]:
## Initialize Test/Train Stock Lists and Check test vs train return
train, test = train_test_split(nasdaqList, test_size=0.3, shuffle=True)
testTrainR = []
def calculateTestTrainRatio(train,test):
    testTrainRatio = [1,1]
    for i in range(2):
        if i == 0:
            l = test
        else:
            l = train
        index = 0
        while index != len(l):
            if l[index] not in listOfDf.columns:
                l = np.delete(l, index)
            else:
                index += 1
        for index, element in listOfDf[l].iterrows():
            #print(element.to_list())
            listOfStockRet = element.to_list()
            while 1.0 in listOfStockRet:
                listOfStockRet.remove(1.0)
            if len(listOfStockRet) != 0:
                testTrainRatio[i] = testTrainRatio[i] * np.mean(listOfStockRet)     
    testTrainRatio[1] = testTrainRatio[1] / testTrainRatio[0]
    testTrainRatio[0] = 1
    return testTrainRatio
testTrainR = calculateTestTrainRatio(train,test)
while(abs(1-testTrainR[1])>0.2):
    train, test = train_test_split(nasdaqList, test_size=0.3, shuffle=True)
    testTrainR = calculateTestTrainRatio(train,test)
    